In [27]:
!pip install scipy
!pip install -U kaleido

In [ ]:
import numpy as np
import datetime as dt
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
import scipy as sc

# going to be importing data

def getData(stocks, start, end):
    data = yf.download(
        stocks,
        start=start,
        end=end,
        progress=False,
        auto_adjust=True
    )
    return data['Close']

stocklist = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'CIPLA']
stock = [s+'.NS' for s in stocklist]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

stockData = getData(stock, startDate, endDate)
returns = stockData.pct_change()
meanReturns = returns.mean()
covMatrix = returns.cov()
print(stockData)
print(returns)
print(meanReturns)
print(covMatrix)

def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))*np.sqrt(252)
    return returns, std

weights = np.array([0.2,0.3,0.1,0.2,0.2])

returns, std = portfolioPerformance(weights, meanReturns, covMatrix)

Ticker         CIPLA.NS  HDFCBANK.NS      INFY.NS  RELIANCE.NS       TCS.NS
Date                                                                       
2024-12-26  1474.795776   883.433655  1852.574341  1211.710571  4028.955566
2024-12-27  1490.676025   887.133667  1861.655518  1216.192749  4024.848389
2024-12-30  1505.319702   877.094360  1851.214600  1205.883789  4019.001221
2024-12-31  1512.839355   874.603027  1825.961792  1210.614868  3957.152832
2025-01-01  1512.938232   879.486938  1828.389893  1216.391846  3974.209473
...                 ...          ...          ...          ...          ...
2025-12-19  1517.099976   985.500000  1638.699951  1565.099976  3282.000000
2025-12-22  1512.900024   987.700012  1689.599976  1575.400024  3324.899902
2025-12-23  1500.699951   996.599976  1668.300049  1570.699951  3310.000000
2025-12-24  1496.300049   997.200012  1663.400024  1558.199951  3319.000000
2025-12-26  1506.000000   992.099976  1656.099976  1559.199951  3280.000000

[251 rows x

In [ ]:
print(round(returns*100,4), round(std*100,4))

5.5794 13.0611


In [ ]:
from pandas._config.config import reset_option
import scipy as sc # Re-adding the import for scipy

def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0.0415):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return -(pReturns - riskFreeRate)/pStd # Negate for minimization to maximize SR


def maxSR(meanReturns, covMatrix, riskFreeRate = 0.0415, constraintSet = (0,1)): # Maximising the Sharpe Ratio by altering the weights of the portfolio
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(negativeSR, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

result = maxSR(meanReturns, covMatrix)
optimal_sharpe_ratio = -result['fun']
optimal_weights = result['x']
print(f"Maximum Sharpe Ratio: {optimal_sharpe_ratio:.4f}")
print(f"Optimal Weights: {optimal_weights}")

Maximum Sharpe Ratio: 1.1949
Optimal Weights: [1.18339545e-16 4.95647944e-02 0.00000000e+00 9.50435206e-01
 4.08258135e-16]


In [16]:
def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)): # minimizing the portfolio variance by altering the weights of the stocks in the portfolio
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    Bound = constraintSet
    bounds = tuple(Bound for asset in range(numAssets))
    result = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

minVarResult = minimizeVariance(meanReturns, covMatrix)
optimal_variance = minVarResult['fun']
optimal_weights = minVarResult['x']
print(f"Minimum Variance: {optimal_variance:.4f}")
print(f"Optimal Weights: {optimal_weights}")

Minimum Variance: 0.1255
Optimal Weights: [0.18507465 0.4639197  0.         0.08174312 0.26926252]


In [13]:
import numpy as np
import datetime as dt
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
import scipy as sc

# Data fetching and preparation
def getData(stocks, start, end):
    data = yf.download(
        stocks,
        start=start,
        end=end,
        progress=False,
        auto_adjust=True
    )
    return data['Close']

stocklist = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'CIPLA']
stock = [s+'.NS' for s in stocklist]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

stockData = getData(stock, startDate, endDate)
returns = stockData.pct_change()
meanReturns = returns.mean()
covMatrix = returns.cov()

# Portfolio Performance functions
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))*np.sqrt(252)
    return returns, std

def portfolioReturn(weights, meanReturns, covMatrix):
    return np.sum(meanReturns*weights)*252

def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0.0415):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return -(pReturns - riskFreeRate)/pStd # Negate for minimization to maximize SR

# Optimization functions
def maxSR(meanReturns, covMatrix, riskFreeRate = 0.0415, constraintSet = (0,1)): # Maximising the Sharpe Ratio by altering the weights of the portfolio
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(negativeSR, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)): # minimizing the portfolio variance by altering the weights of the stocks in the portfolio
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    Bound = constraintSet
    bounds = tuple(Bound for asset in range(numAssets))
    result = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def EfficientOptimization(meanReturns, covMatrix, returnTarget, constraintSet = (0,1)): # For each target return we want to optimise the portfolio for min variance
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                   {'type':'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple(constraintSet for asset in range(numAssets))
    effOpt = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

# Original calculatedResults function
def calculatedResults(meanReturns, covMatrix, riskFreeRate = 0.0415, constraintSet = (0,1)):
  # for Max Sharpe Ratio Portfolio
    optimalResult = maxSR(meanReturns, covMatrix, riskFreeRate, constraintSet)
    maxSR_returns, maxSR_std = portfolioPerformance(optimalResult['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    maxSR_allocation = pd.DataFrame(optimalResult['x'], index=stockData.columns, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]
  # For Min volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=stockData.columns, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]
# Efficient Frontier
    efficientlist = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
      efficientlist.append(EfficientOptimization(meanReturns, covMatrix, target/100)['fun'])

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientlist, targetReturns


print(calculatedResults(meanReturns, covMatrix))


(np.float64(26.59), np.float64(19.05),              allocation
Ticker                 
CIPLA.NS            0.0
HDFCBANK.NS         3.0
INFY.NS             0.0
RELIANCE.NS        97.0
TCS.NS              0.0, np.float64(3.54), np.float64(12.57),              allocation
Ticker                 
CIPLA.NS           19.0
HDFCBANK.NS        46.0
INFY.NS             0.0
RELIANCE.NS         8.0
TCS.NS             27.0, [np.float64(0.12565155714769144), np.float64(0.12578548866326997), np.float64(0.12618396270716525), np.float64(0.12684393331949217), np.float64(0.1277630928115976), np.float64(0.12893569041679775), np.float64(0.13035495768277766), np.float64(0.132012804450403), np.float64(0.13389954433100448), np.float64(0.13600653658799364), np.float64(0.13831855966165602), np.float64(0.14081981187030684), np.float64(0.14368439545174896), np.float64(0.1472023395548307), np.float64(0.15139298229763043), np.float64(0.15681066170688768), np.float64(0.16358021146879553), np.float64(0.17154165949977)

In [7]:
import numpy as np
import datetime as dt
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
import scipy as sc

# --- Code to define meanReturns and covMatrix (copied from epKpFXZClS8J to resolve NameError) ---
def getData(stocks, start, end):
    data = yf.download(
        stocks,
        start=start,
        end=end,
        progress=False,
        auto_adjust=True
    )
    return data['Close']

stocklist = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'CIPLA']
stock = [s+'.NS' for s in stocklist]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

stockData = getData(stock, startDate, endDate)
returns = stockData.pct_change()
meanReturns = returns.mean()
covMatrix = returns.cov()
# --------------------------------------------------------------------------------------------------

def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))*np.sqrt(252)
    return returns, std

def portfolioReturn(weights, meanReturns, covMatrix):
    return np.sum(meanReturns*weights)*252

def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def EfficientOptimization(meanReturns, covMatrix, returnTarget, constraintSet = (0,1)): # For each target return we want to optimise the portfolio for min variance
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                   {'type':'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(constraintSet for asset in range(numAssets))
    effOpt = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

print(EfficientOptimization(meanReturns, covMatrix, 0.05))

     message: Optimization terminated successfully
     success: True
      status: 0
         fun: 0.1258458097018032
           x: [ 1.801e-01  4.695e-01  3.144e-18  1.122e-01  2.382e-01]
         nit: 6
         jac: [ 1.258e-01  1.281e-01  1.360e-01  1.312e-01  1.189e-01]
        nfev: 36
        njev: 6
 multipliers: [ 2.648e-02  1.245e-01]


In [13]:
import numpy as np
import datetime as dt
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
import scipy as sc
import plotly.graph_objects as go
import kaleido # Import kaleido to use its function

# Data fetching and preparation
def getData(stocks, start, end):
    data = yf.download(
        stocks,
        start=start,
        end=end,
        progress=False,
        auto_adjust=True
    )
    return data['Close']

stocklist = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'CIPLA']
stock = [s+'.NS' for s in stocklist]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

stockData = getData(stock, startDate, endDate)
returns = stockData.pct_change()
meanReturns = returns.mean()
covMatrix = returns.cov()

# Portfolio Performance functions
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))*np.sqrt(252)
    return returns, std

def portfolioReturn(weights, meanReturns, covMatrix):
    return np.sum(meanReturns*weights)*252

def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0.0415):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return -(pReturns - riskFreeRate)/pStd # Negate for minimization to maximize SR

# Optimization functions
def maxSR(meanReturns, covMatrix, riskFreeRate = 0.0415, constraintSet = (0,1)): # Maximising the Sharpe Ratio by altering the weights of the portfolio
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(negativeSR, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)): # minimizing the portfolio variance by altering the weights of the stocks in the portfolio
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    Bound = constraintSet
    bounds = tuple(Bound for asset in range(numAssets))
    result = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def EfficientOptimization(meanReturns, covMatrix, returnTarget, constraintSet = (0,1)): # For each target return we want to optimise the portfolio for min variance
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                   {'type':'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple(constraintSet for asset in range(numAssets))
    effOpt = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

# calculatedResults function
def calculatedResults(meanReturns, covMatrix, riskFreeRate = 0.0415, constraintSet = (0,1)):
  # for Max Sharpe Ratio Portfolio
    optimalResult = maxSR(meanReturns, covMatrix, riskFreeRate, constraintSet)
    maxSR_returns, maxSR_std = portfolioPerformance(optimalResult['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    maxSR_allocation = pd.DataFrame(optimalResult['x'], index=stockData.columns, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]
  # For Min volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=stockData.columns, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]
# Efficient Frontier
    efficientlist = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
      efficientlist.append(EfficientOptimization(meanReturns, covMatrix, target/100)['fun'])

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientlist, targetReturns


def EF_graph(meanReturns, covMatrix, riskFreeRate = 0.0415, constraintSet = (0,1)): # returning a graph the min volatility, max SR and efficient frontier
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientlist, targetReturns = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet)

    #Max SR
    MaxSharpeRatio = go.Scatter(x=[maxSR_std], y=[maxSR_returns], mode='markers', name='Max Sharpe Ratio', marker=dict(color='red', size=14, line=dict(width=3, color='black')))

    #Min Volatilty
    MinVolatility = go.Scatter(x=[minVol_std], y=[minVol_returns], mode='markers', name='Min Volatility', marker=dict(color='green', size=14, line=dict(width=3, color='black')))

    #Efficient Frontier
    EF_curve = go.Scatter(x=[round(i*100,2) for i in efficientlist], y=targetReturns, mode='lines', name='Efficient Frontier', line=dict(color='black', width=4, dash='dot'))

    data = [MaxSharpeRatio, MinVolatility, EF_curve]
    layout = go.Layout(title='Portfolio Optimization With Efficient Frontier', xaxis=dict(title='Volatility (%)'), yaxis=dict(title='Return (%)'), showlegend=True, legend=dict(x=0.75, y=0, traceorder='normal', bgcolor='#E2E2E2', bordercolor='black', borderwidth=2))
    fig = go.Figure(data=data, layout=layout)
    fig.show()
    return fig

kaleido.get_chrome_sync()
fig = EF_graph(meanReturns, covMatrix)
fig.write_image("efficient_frontier.png")
from google.colab import files
files.download("efficient_frontier.png")





<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
!sudo apt update && sudo apt-get install libnss3 libatk-bridge2.0-0 libcups2 libxcomposite1 libxdamage1 libxfixes3 libxrandr2 libgbm1 libxkbcommon0 libpango-1.0-0 libcairo2 libasound2

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,966 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,572 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:14 ht